In [1]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime, timedelta

In [4]:
hubs = ['ALBE', 'ALDE', 'AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

start = datetime(2022, 9, 1).strftime('%Y-%m-%d')
end = datetime(2023, 3, 31).strftime('%Y-%m-%d')

In [5]:
def retrieve_net_positions(fromDate, toDate):
    data = {
        'fromUtc': f'{fromDate}T00:00:00.000Z',
        'toUtc': f'{toDate}T23:00:00.000Z',
    }

    try:
        response = requests.post('https://publicationtool.jao.eu/core/api/data/netPos', json=data)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        
    return pd.DataFrame(response.json()['data'])

In [9]:
def retrieve_ptdf(fromDate, toDate):
    data = {
        'fromUtc': f'{fromDate}T00:00:00.000Z',
        'toUtc': f'{toDate}T23:00:00.000Z',
    }

    try:
        response = requests.post('https://publicationtool.jao.eu/core/api/data/preFinalComputation', json=data)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
   
    return pd.DataFrame(response.json()['data'])

In [7]:
# NET POSITIONS
df_netpos = retrieve_net_positions(start, end)

df_netpos = df_netpos.rename(columns={
    'dateTimeUtc': 'DateTime'
})
df_netpos = df_netpos.set_index('DateTime')
df_netpos.index = pd.to_datetime(df_netpos.index)
df_netpos = df_netpos.tz_localize(None)

df_netpos = df_netpos[[
    'hub_ALBE', 'hub_ALDE', 'hub_AT', 'hub_BE', 'hub_CZ', 'hub_DE', 'hub_FR', 'hub_HR', 'hub_HU', 'hub_NL', 
    'hub_PL', 'hub_RO', 'hub_SI', 'hub_SK'
]]

df_netpos.columns = hubs

df_netpos.to_excel("./data/net_positions.xlsx")

In [10]:
# ZONAL PTDF and RAM
df_ptdf = retrieve_ptdf(start, end)

df_ptdf = df_ptdf.rename(columns={
    'dateTimeUtc': 'DateTime'
})
df_ptdf = df_ptdf.set_index('DateTime')
df_ptdf.index = pd.to_datetime(df_ptdf.index)
df_ptdf = df_ptdf.tz_localize(None)

df_ptdf = df_ptdf[df_ptdf.cneEic != 'NA']
df_ptdf = df_ptdf[[
    'cneEic', 'ram', 'ptdf_ALBE', 'ptdf_ALDE', 'ptdf_AT', 'ptdf_BE', 'ptdf_CZ', 'ptdf_DE', 'ptdf_FR', 'ptdf_HR', 'ptdf_HU', 'ptdf_NL', 
    'ptdf_PL', 'ptdf_RO', 'ptdf_SI', 'ptdf_SK'
]]

df_ptdf.columns = ['eic', 'ram'] + hubs

df_ptdf.to_excel("./data/ptdfs.xlsx")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

